# Test Jupyter Notebook
Test code to move MRP around with rospy.

## Moving MRP around

In [4]:
import rospy
from geometry_msgs.msg import Twist

In [5]:
rospy.init_node('test_node', anonymous=True)

In [13]:
pub = rospy.Publisher('/tb_cmd_vel', Twist, queue_size=10)

message = Twist()
message.linear.x = 0.2

pub.publish(message)

In [16]:
message.linear.x = 0
message.angular.z = 0.3

pub.publish(message)

In [17]:
message.linear.x = 0
message.angular.z = 0

pub.publish(message)

## Moving human actor around